In [4]:
import requests
import time 
from bs4 import BeautifulSoup
#from mechanize import Browser
import pandas as pd
from IPython.display import HTML

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time

from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

In [5]:
!conda install -c conda-forge -y wordcloud

Jupyter detected...
2 channel Terms of Service accepted
Channels:
 - conda-forge
 - defaults
Platform: osx-64
doneecting package metadata (repodata.json): - 
doneing environment: - 


==> WARNING: A newer version of conda exists. <==
    current version: 25.7.0
    latest version: 25.9.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



In [8]:
# Gets the first page

# Specify the search
area = "Data Science"
location = "California"
web_address='https://www.indeed.com/jobs?q='+area+'&l='+location

options = Options()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--remote-debugging-port=9222")

# Annoyingly I had to specify an exact driver version
# Driver manager should figure this out for me, but it didn't
# You should be able to just run: service = Service(ChromeDriverManager().install())
service = Service(ChromeDriverManager(driver_version="140.0.7339.185").install())
driver = webdriver.Chrome(service=service,options=options)

driver.get(web_address)
html_source_code = driver.execute_script("return document.body.innerHTML;")

# Save to beautiful soup
soup = BeautifulSoup(html_source_code, 'html.parser')

# Close page
driver.close()

SessionNotCreatedException: Message: session not created
from unknown error: cannot find Chrome binary
Stacktrace:
0   chromedriver                        0x000000010f71a908 chromedriver + 6379784
1   chromedriver                        0x000000010f71200a chromedriver + 6344714
2   chromedriver                        0x000000010f16ddf0 chromedriver + 429552
3   chromedriver                        0x000000010f1a6049 chromedriver + 659529
4   chromedriver                        0x000000010f1a4c45 chromedriver + 654405
5   chromedriver                        0x000000010f1f44b1 chromedriver + 980145
6   chromedriver                        0x000000010f1f3b46 chromedriver + 977734
7   chromedriver                        0x000000010f1e5ea3 chromedriver + 921251
8   chromedriver                        0x000000010f1b239a chromedriver + 709530
9   chromedriver                        0x000000010f1b3061 chromedriver + 712801
10  chromedriver                        0x000000010f6d87b0 chromedriver +

In [9]:
# Get the names of the jobs
result2 = soup.find_all('h2', class_="jobTitle")
job_names = [r.text for r in result2]
print(len(job_names))
job_names

NameError: name 'soup' is not defined

In [10]:
# List of indeed links to the named jobs
result3 = soup.find_all('a', class_="jcs-JobTitle")
links = ['https://www.indeed.com' + r.get('href') for r in result3]
print(len(links))
links

NameError: name 'soup' is not defined

In [11]:
# I want to loop through each of the links above and get the full job descriptions
data = []

for link in links:
    driver = webdriver.Chrome(service=service,options=options)
    driver.get(link)

    html_source_code = driver.execute_script("return document.body.innerHTML;")

    # Save to beautiful soup
    soup_tmp = BeautifulSoup(html_source_code, 'html.parser')
    driver.close()
    
    job_info = soup_tmp.find_all('div', id="jobDescriptionText")
    for j in job_info:
        data.append(j.text)

NameError: name 'links' is not defined

## Make WordCloud From the Data

In [12]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

#df = pd.read_csv(r"Youtube04-Eminem.csv", encoding ="latin-1")
 
comment_words = ''
stopwords = set(STOPWORDS)
ADDWORDS = {'work',"need","location","working",
            "employee","will","benefits","compensation",
           "must", "may","company",'candidate','position','job', 'required','provide','including'
           "application",'employees','product','customer','including','San Francisco,''application',
           'planning','org','year','requirement','water','supply','application','years','project',
           'requirements','applicant','related','san','francisco','Experience'}
stopwords.update(ADDWORDS)
 
# iterate through the csv file
for val in data:
     
    # typecaste each val to string
    val = str(val)
 
    # split the value
    tokens = val.split()
     
    # Converts each token into lowercase
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()
     
    comment_words += " ".join(tokens)+" "
 
wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                stopwords = stopwords,
                min_font_size = 10).generate(comment_words)
 
# plot the WordCloud image                      
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)

name = "WordCloud_JobNames_" + '_search_' + area + '_' + location + '_' + '.png'
wordcloud.to_file(name)
 
plt.show()


ValueError: We need at least 1 word to plot a word cloud, got 0.

## Now do what we just did but gather data from multiple pages

In [ ]:
# THis takes a while and web browser will keep opening and closing
#---------- pass variables ---------------
area = "Data Science"
location = "California"
pages=8
removeNA=False
bachelors_only=True
min_salary = 50000

#---------- def function ----------------
JOBS = []

companies = []
cities = []
salaries = []
job_titles = []
links = []

sc = '&vjk=44c2856b496520b7'
for page in range(pages):
    if page ==0:
        web_address='https://www.indeed.com/jobs?q='+area + '+$' + str(min_salary) +'&l='+location
        if bachelors_only == True:
            web_address='https://www.indeed.com/jobs?q='+area+ '+$' + str(min_salary) +'&l='+location + sc  
    else:
        start_page = page*10
        web_address='https://www.indeed.com/jobs?q='+area+ '+$' + str(min_salary) +'&l='+location+'&start='+str(start_page)
        if bachelors_only == True:
            web_address='https://www.indeed.com/jobs?q='+area+ '+$' + str(min_salary) +'&l='+location + sc +'&start='+str(start_page)
            
    
    # Get the page data
    driver = webdriver.Chrome(service=service,options=options)
    driver.get(web_address)
    # wait a few seconds for the page to load
    time.sleep(3)
    html_source_code = driver.execute_script("return document.body.innerHTML;")
    soup = BeautifulSoup(html_source_code, 'html.parser')
    # Close page
    driver.close()
    
    # Save the job info Data Frame
    
    # Job Titles
    job_results = soup.find_all('h2', class_="jobTitle")
    jobs = [r.text for r in job_results]
    for j in jobs:
        job_titles.append(j)
    
    # Links to the named jobs
    link_results = soup.find_all('a', class_="jcs-JobTitle")
    tmp_links = ['https://www.indeed.com' + r.get('href') for r in link_results]
    for l in tmp_links:
        links.append(l)
    
    # Get company and salary info
    info_results = soup.find_all('td', class_="resultContent css-1o6lhys eu4oa1w0")
    for i in range(len(info_results)):
        tmp_company = info_results[i].find_all("span", {"data-testid": "company-name"})
        if tmp_company == []:
            tmp_company = 'NONE'
            companies.append(tmp_company)
        else:
            companies.append(tmp_company[0].text)
        
        tmp_location = info_results[i].find_all('div', {"data-testid": "text-location"})
        if tmp_location == []:
            tmp_location = 'NONE'
            cities.append(tmp_location)
        else:
            cities.append(tmp_location[0].text)
        tmp_salary = info_results[i].find_all('h2', class_="mosaic-provider-jobcards-5vqdjd e1tiznh50")
        if tmp_salary == []:
            tmp_salary = 'NA'
            salaries.append(tmp_salary)
        else:
            salaries.append(tmp_salary[0].text)
    
    
    for i, job in enumerate(job_titles): 
        company = companies[i]
        city = cities[i]
        salary = salaries[i]
        link = links[i]
        tmp = (job,
               company, 
               city,
               salary,
               link)
        JOBS.append(tmp)

# Create the results Data Frame
job_listings = pd.DataFrame(JOBS, 
                            columns=['Job',
                                     'Company',
                                     'Location',
                                     'Salary',
                                     'Link'],
                           index=[i+1 for i in range(len(JOBS))])

total_list = len(job_listings)
job_listings = job_listings.drop_duplicates(subset='Link')
unique_len = len(job_listings)
print("Removed {} duplicates".format(total_list-unique_len))


print("In the first {} pages: Found {} unique jobs matching {} in {} with a minimum salary {}".format(pages,job_listings.shape[0],area,location,min_salary))
if bachelors_only == True:
    print("Eductation = Bachelors Degree")
if removeNA==True:
    print("Removing jobs with NA salary...")
    job_listings = job_listings[job_listings.Salary!='NA'].copy().reset_index()
    print("Resulting in {} remaining jobs".format(job_listings.shape[0]))
    


job_listings

In [ ]:
file_name = "WordCloud_JobNames_" + str(len(list(job_listings.Job))) + '_search_' + area + '_' + location + '_' + str(min_salary) + '.xlsx'
job_listings.to_excel(file_name)

## Make word cloud of job titles

In [ ]:
DATA = list(job_listings.Job)

print("Analyzing {} jobs from Indeed.com with search: \n Subject Area = {} \n Location = {} \n Salary {}".format(len(DATA),area,location,min_salary))
   
comment_words = ''
stopwords = set(STOPWORDS)
ADDWORDS = {'work',"need","location","working",
            "employee","will","benefits","compensation",
           "must", "may","company",'candidate','position','job'}
stopwords.update(ADDWORDS)

for val in DATA:
    val = str(val)
    tokens = val.split()
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()

    comment_words += " ".join(tokens)+" "

wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                stopwords = stopwords,
                min_font_size = 10).generate(comment_words)

# plot the WordCloud image                      
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)

plt.show()
name = "WordCloud_JobNames_" + str(len(list(job_listings.Job))) + '_search_' + area + '_' + location + '_' + str(min_salary) + '.png'
wordcloud.to_file(name)